# Measurements

In [ ]:
import polars as pl
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.graph_objects as go

In [ ]:
pio.templates.default = "none"

In [ ]:
# data = (
#     pl.read_csv("measurement.csv", has_header=False, new_columns=["at", "temperature", "humidity"])
#     .with_columns(pl.col("at").str.to_datetime("%F %T%.9f %Z", time_zone="UTC"))
# )

data = (
    pl.read_database_uri("select at, temperature, humidity from measurement", "sqlite://measurement.sqlite")
    .with_columns(pl.col("at").cast(pl.Datetime("ms", "UTC")))
    .group_by_dynamic("at", every="5m")
    .agg(pl.all().mean())
)
data

In [ ]:
fig = make_subplots(rows=2, subplot_titles=["Temperature", "Humidity"], shared_xaxes=True)

for i, col in enumerate(["temperature", "humidity"]):
    fig.add_trace(go.Scatter(x=data["at"], y=data[col], name=col), row=i+1, col=1)

fig.update_layout(title="Sensor Measurements", height=800)
fig